# Characteristic
A point cloud is a set of points with coordinates, and different from an image and meshes. In this section, we introduce the following characteristics of the point cloud. 
- Properties of points
- Point density
- Invariance to rigid transformation
- Unordered points
- Definition of Neighborhood


## Properties of points
Points have coordinats and the coordinates are XYZ values. Points have coordinates, and the coordinates are XYZ values. The point position depends on the coordinates.

Also, points can have colors and normals, etc.
- **Color (RGB)**: the color is used when displaying points on viewers, or for point cloud processing.
- **Normals**: we can get the surface of a 3D object or detect planes from normals.
- **Intensity**:  "intensity" is the laser reflection intensity, and depends on the reflectivity of the object surface that the laser hits. Intensity can be used for feature extraction.


## Point density
There are density differences of points in a point cloud because points are not arranged regularly. An example of a point density difference is a point distribution acquired from a sensor. You can confirm a point cloud acquired from a sensor by following code:


In [1]:
from tutlibs.io import Points
from tutlibs.visualization import JupyterVisualizer as jv
coords, colors, _ = Points.read("../data/kitti_sample.ply")
    obj_points = jv.point(coords, colors, point_size=0.07)
jv.display([obj_points])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Output()

The point colors in outputs show the distance from the sensor to the point, and red is the closest. Let's compare the surrounding of the orange and blue points. You can see that the orange point has a dense point around it, while the blue point is sparse. The density differences of points affect point cloud processing such as nearest neighbor search, downsampling.


## Invariance to rigid transformations
Point cloud shape is invariance to rotation and translation (rigid transformation). In some tasks, we must consider rotation and translation of a point cloud. For example, in the classification task, we must prepare a method that is invariant to an object's direction because input object data do not have a consistent direction.



## Unordered points
Points on a point cloud do not depend on order. Therefore, a point cloud shape is immutable even if points are reordered. We can confirm this characteristic by the following code:

In [7]:
import numpy as np
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color
from tutlibs.transformation import Transformation as tr
from tutlibs.io import Points as io

In [8]:
coords, _, _ = io.read('../data/bunny_pc.ply')
num_points = len(coords)

# reorder
idxs = np.random.choice(np.arange(num_points), num_points, replace=False)
reordered_coords = coords[idxs]

# translation to compare an origin
reordered_coords = tr.translation(reordered_coords, np.array([1, 0, 0]))

obj_points = jv.point(coords, single_color("#ff0000", num_points))
obj_random_points = jv.point(reordered_coords, single_color("#00ff00", num_points))
jv.display([obj_points, obj_random_points])

Output()

The above output shows the original point cloud (green) and the reordered point cloud (red). As you can see, the point shape have no change. If we do not use point indices obtained from a depth camera image, LiDAR, etc., it is necessary to consider processing that does not depend on point order.


## Definition of Neighborhood
A point cloud does not have adjacencies between points. In the point cloud processing, we might need to define adjacency to use convolutions and handcrafted features, etc. We can access adjacencies of pixels by a grid index, on the other hand, we need to define adjacencies of points from point coordinates and distances. This adjacency depends on the nearest neighbor search. For example, kNN (k Nearest Neighbors) and radius nearest neighbor define neighborhoods by the number of points (k) and range (r), respectively, so adjacencies are different. We show kNN and radius nearest neighbors (red is neighbors, blue is other):

![neighbors](img/neighbors.png)

The definition make the following difference.
- If we get k neighbors from N query points with kNN, kNN output an array with ($N \times k$). However, we get far points if there are no points close to queries.
- We can get neighbors within r (radius) by radius nearest neighbor. However, the number of neighbors may be different by queries.

As shown in the above example, the definition of neighbors has a significant impact on the subsequent processing.


# Reference

